In [18]:
import numpy as np
import math
import cv2
import tdmclient.notebook
import utils

In [8]:
await tdmclient.notebook.start(tdm_addr='172.25.112.1', tdm_port=8596)

In [7]:
# await tdmclient.notebook.stop()

In [11]:
%%run_aseba
onevent run
    motor.left.target = 300
    motor.right.target = 300

onevent turnleft
    motor.left.target = -300
    motor.right.target = 300

onevent turnright
    motor.left.target = 300
    motor.right.target = -300

onevent stop
    motor.left.target = 0
    motor.right.target = 0

Define possible objects and marker colors

In [ ]:
robot1 = {"front":1, "back":2, "f_rgb": [136, 0, 20], "b_rgb": [3, 161, 234]}
robot2 = {"front":3, "back":4, "f_rgb": [23, 255, 22], "b_rgb": [163, 73, 164]}
ball = {"center":5, "rgb": [254, 242, 0]}
goal1 = {"center": 7, "rgb": [100, 100, 100]}
goal2 = {"center": 6, "rgb": [195, 195, 195]}

In [ ]:
field = {
    "ball": ball,
    "goal1": goal1,
    "goal2": goal2,
}

Define what robots are in the game

In [ ]:
robots = {
    "robot1": robot1,
    "robot2": robot2
}


Variables

In [19]:
COLOR_TOLERANCE: int = 20
DIST_BALL: int = 10

In [20]:
cap = cv2.VideoCapture(0)

[ WARN:0@1946.063] global cap_v4l.cpp:982 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@1946.063] global obsensor_uvc_stream_channel.cpp:156 getStreamChannelGroup Camera index out of range


In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
        break
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    matrix = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)

    # Process each robot's front and back points
    for robot, details in robots.items():
        # Detect front points
        f_lower_bound = np.array(details['f_rgb']) - COLOR_TOLERANCE
        f_upper_bound = np.array(details['f_rgb']) + COLOR_TOLERANCE
        front_coordinates = utils.detect_colored_dots_in_rgb(image, f_lower_bound, f_upper_bound)
        
        # Mark front points on the matrix
        for coord in front_coordinates:
            matrix[coord[1], coord[0]] = details['front']
        
        # Detect back points
        b_lower_bound = np.array(details['b_rgb']) - COLOR_TOLERANCE
        b_upper_bound = np.array(details['b_rgb']) + COLOR_TOLERANCE
        back_coordinates = utils.detect_colored_dots_in_rgb(image, b_lower_bound, b_upper_bound)
        
        # Mark back points on the matrix
        for coord in back_coordinates:
            matrix[coord[1], coord[0]] = details['back']

    # Find the coordinates of goals and ball
    for feature, details in field.items():
        lower_bound = np.array(details['rgb']) - COLOR_TOLERANCE
        upper_bound = np.array(details['rgb']) + COLOR_TOLERANCE
        coordinates = utils.detect_colored_dots_in_rgb(image, lower_bound, upper_bound)
        for coord in coordinates:
            matrix[coord[1], coord[0]] = details['center']
    

    #___________________________________________________________TEST_PART______________________________________

    color_map = {
        0: (0, 0, 0),       # Black (assuming 0 is the background or no dot)
        1: (255, 0, 0),     # Red
        2: (0, 255, 0),     # Green
        3: (0, 0, 255),     # Blue
        4: (255, 255, 0),   # Cyan
        5: (255, 0, 255),   # Magenta
        6: (0, 255, 255),   # Yellow
        7: (255, 255, 255)  # White
    }

    # Create an empty frame with 3 color channels
    colored_frame = np.zeros((matrix.shape[0], matrix.shape[1], 3), dtype=np.uint8)

    # Draw colored circles in the frame based on matrix values
    for y in range(matrix.shape[0]):
        for x in range(matrix.shape[1]):
            value = matrix[y, x]
            if value in color_map:
                cv2.circle(colored_frame, (x, y), 10, color_map[value], -1)  # Circle with radius 10

    # Display the frame
    cv2.imshow('Frame', colored_frame)

    # Wait for 'q' key to quit
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    #___________________________________________________________TEST_PART______________________________________
    
cap.release()
cv2.destroyAllWindows()